In [ ]:
# run once (may have to restart notebook)
# !pip install tensorflow-gpu==1.15 --user # if you do not have a gpu remove -gpu 
# !pip install gpt-2-simple --user

In [1]:
import pandas as pd
import numpy as np
import json
import os
import requests

In [2]:
is_local = False # change this if you are not loading a pretrained model locally
if is_local:
    # import tensorflow as tf
    import sys
    sys.path.insert(0, os.path.abspath('../../gpt-2-simple-0.7/gpt_2_simple'))
    import gpt_2 as gpt2
    local_checkpoint_dir = "../../local_checkpoints" # directory where local models are stored
    local_model_name = 'model-100'
else:
    import gpt_2_simple as gpt2

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
# check to make sure gpu is recognized for significantly faster training

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [4]:
model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)

In [5]:
dir_path = "../data/matt_results/"

dem_file_name = "democrats_result.txt"
dem_model_name = 'creighton'

rep_file_name = 'republican_result.txt'
rep_model_name = 'carlisle'

both_file_name = 'both_result.txt'
both_model_name = 'kuzco'

In [14]:
# text file to train model on
train_fp = dir_path + rep_file_name
train_name = rep_model_name
results_fp = "../results/" + train_name + "_generated_0.txt"

In [7]:
# this cell takes the longest. Can only be run once without restarting the notebook
sess = gpt2.start_tf_sess()
if is_local:
    gpt2.load_gpt2(sess, 
                   checkpoint=local_model_name, 
                   run_name=train_name, 
                   checkpoint_dir=local_checkpoint_dir)
else:
    gpt2.finetune(sess,
                  train_fp,
                  model_name=model_name,
                  steps=100, # steps is max number of training steps
                  restore_from='fresh', # makes sure model doesnt resume from previous trained model
                  print_every=20, # only prints every 20 training steps,
                  run_name=train_name # model name, so we can load different models locally
                 )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.59s/it]


dataset has 259398 tokens
Training...
[20 | 25.28] loss=3.08 avg=3.08
[40 | 44.62] loss=3.17 avg=3.12
[60 | 64.10] loss=2.75 avg=3.00
[80 | 83.59] loss=2.64 avg=2.91
[100 | 102.87] loss=2.91 avg=2.91
Saving checkpoint/carlisle/model-100


In [15]:
# prompt to generate response to, going to be a post/comment from the political discussion subreddits

pre = """
[title]
You say: "Police violence is problematic." - They hear: "I am fine with looting and arson." - You say: "I want criminal arsonists arrested." - They hear: "I want cops to break up peaceful protests and beat them up."

[selftext]
Just a quick guide to what the other party understands from your positions. For your discussions and debates on this sub and elsewhere. I didn't come up with it, I merely translated it from memory. Can't find the original source, sorry.
"""

In [16]:
# gpt2.generate(sess, 
#               temperature=.7, # uniqueness of the output (usually ranges from .5 to 2)
#               prefix=pre, # prompt
#               nsamples=5, # number of generated responses 
#               length=1000 # number of words (including prompt) per response
#              )

In [17]:
gpt2.generate_to_file(sess, 
                      destination_path=results_fp,
                      temperature=.7, # uniqueness of the output (usually ranges from .5 to 2)
                      prefix=pre, # prompt
                      nsamples=50, # number of generated responses 
                      length=1000 # number of words (including prompt) per response)
                     )